<a href="https://colab.research.google.com/github/h672797/DAT158ML2/blob/main/catordog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


Extract Zip

In [2]:
import zipfile

zip_path = list(uploaded.keys())[0]  # Get the uploaded zip filename

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')  # Extract into a folder named 'dataset'

print("Extraction complete!")

Extraction complete!


Check Files

In [3]:
import os
print(os.listdir('dataset'))

['PetImages']


Import


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import numpy as np

Load Data

In [20]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

import os
from PIL import Image, UnidentifiedImageError
import shutil

base_dir = 'dataset/PetImages'
categories = ['Cat', 'Dog']

# Create a clean folder to copy only valid RGB images
clean_dir = 'dataset_clean/PetImages'
os.makedirs(clean_dir, exist_ok=True)

for category in categories:
    src_folder = os.path.join(base_dir, category)
    dst_folder = os.path.join(clean_dir, category)
    os.makedirs(dst_folder, exist_ok=True)

    for filename in os.listdir(src_folder):
        file_path = os.path.join(src_folder, filename)
        try:
            img = Image.open(file_path)
            img = img.convert('RGB')  # Force RGB
            img.save(os.path.join(dst_folder, filename))
        except (IOError, SyntaxError, UnidentifiedImageError):
            print("Skipping invalid image:", file_path)

train_ds = image_dataset_from_directory(
    'dataset_clean/PetImages',
    image_size=(200, 200),
    validation_split=0.1,
    subset='training',
    seed=42,
    batch_size=32
)

val_ds = image_dataset_from_directory(
    'dataset_clean/PetImages',
    image_size=(200, 200),
    validation_split=0.1,
    subset='validation',
    seed=42,
    batch_size=32
)

Found 24998 files belonging to 2 classes.
Using 22499 files for training.
Found 24998 files belonging to 2 classes.
Using 2499 files for validation.


CNN

In [21]:
from tensorflow.keras import Input

model = tf.keras.models.Sequential([
    Input(shape=(200, 200, 3)),  # <-- Use Input layer instead of input_shape in Conv2D
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,889 (429.25 KB)

 Trainable params: 109,889 (429.25 KB)

 Non-trainable params: 0 (0.00 B)

Filter out bad images

In [22]:
import os
from PIL import Image, UnidentifiedImageError

base_dir = 'dataset/PetImages'
categories = ['Cat', 'Dog']

removed_count = 0
for category in categories:
    folder = os.path.join(base_dir, category)
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            img = Image.open(file_path)
            img.verify()  # Check for corruption
        except (IOError, SyntaxError, UnidentifiedImageError):
            print('Removing corrupted image:', file_path)
            os.remove(file_path)
            removed_count += 1

print(f"Removed {removed_count} corrupted images.")

Removed 0 corrupted images.


Training Model

In [23]:
history = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1715s 2s/step - accuracy: 0.5621 - loss: 1.0330 - val_accuracy: 0.6359 - val_loss: 0.6289
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1711s 2s/step - accuracy: 0.6552 - loss: 0.6190 - val_accuracy: 0.6911 - val_loss: 0.5783
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1700s 2s/step - accuracy: 0.7005 - loss: 0.5738 - val_accuracy: 0.7531 - val_loss: 0.5317
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1710s 2s/step - accuracy: 0.7316 - loss: 0.5410 - val_accuracy: 0.7403 - val_loss: 0.5316
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 1759s 2s/step - accuracy: 0.7430 - loss: 0.5293 - val_accuracy: 0.7819 - val_loss: 0.4700


Image Prediction

In [36]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

Saving 3408.webp to 3408.webp


In [40]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(200, 200))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    result = model.predict(img_array)[0][0]
    print("Prediction:", "Dog" if result > 0.5 else "Cat")

predict_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Prediction: Cat
